In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
# glove embedding
!kaggle competitions download -c quora-insincere-questions-classification

  0% 0.00/4.09M [00:00<?, ?B/s]
100% 4.09M/4.09M [00:00<00:00, 57.9MB/s]
 38% 6.00M/15.8M [00:00<00:00, 60.5MB/s]
100% 15.8M/15.8M [00:00<00:00, 101MB/s] 
 86% 47.0M/54.9M [00:00<00:00, 69.9MB/s]
100% 54.9M/54.9M [00:00<00:00, 112MB/s] 
100% 5.95G/5.96G [01:12<00:00, 82.3MB/s]
100% 5.96G/5.96G [01:12<00:00, 88.0MB/s]


In [3]:
!unzip /content/embeddings.zip > /dev/null

# Exploring and Cleaning the data

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('train_aug_in_kag_org.csv')

In [3]:
data.head(10)

,title,text,title_text,label
0,squirrel financial wellbeing,A Squirrel account makes it easy to stick to your budget and save. Start for free today!,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,senior insurance marketing,"Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.","senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,3i financial group inc.,"A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more","3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,brownstone insurance,"Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.","brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending,Learn more about Pacificwide Lending and our commercial & residential loan services in California.,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
5,city auto finance,"City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.","city auto finance. City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
6,"veterans financial, inc.","VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost about benefits","veterans financial, inc.. VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost 

In [4]:
data.shape

(34953, 4)

In [5]:
data.isnull().sum()

title         42
text           0
title_text     0
label          0
dtype: int64

In [6]:
data.dropna(inplace=True)

In [7]:
data.drop(columns=['title', 'text'], inplace=True)

In [8]:
data.head()

,title_text,label
0,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,"senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,"3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,"brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing


# Preparing our data for modeling 

In [9]:
# import re
# import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

In [10]:
# # text
# text_corpus = []
# lm = WordNetLemmatizer()
# words = stopwords.words('english')

# for i in range(data.shape[0]):
#     row = re.sub('[^a-zA-Z\s]', '', data.iloc[i, 0]).lower()
#     row = row.split()  
#     row = [lm.lemmatize(word) for word in row if word not in words]
#     row = ' '.join(row)
#     text_corpus.append(row)

In [11]:
# len(text_corpus)

In [12]:
# text_corpus[0]

_Our Bag of Words_

In [13]:
# X = np.array(text_corpus)
X = data.iloc[:, 0].values
y = data.iloc[:, 1].values

In [14]:
X.shape, y.shape

((34911,), (34911,))

In [15]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [16]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

# Preparing for modeling 

In [17]:
from tqdm import tqdm

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [18]:
token = Tokenizer()

In [19]:
token.fit_on_texts(X)
seq = token.texts_to_sequences(X)

In [20]:
len(token.word_index)

58761

In [21]:
vocab_size = len(token.word_index)+1    #OOV token

In [22]:
l_seq = map(lambda x: len(x), seq)
max(l_seq)

1791

In [23]:
pad_seq = pad_sequences(seq, maxlen=1791)

In [24]:
# Splitting the data
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(pad_seq, y, test_size = 0.2, random_state = 42)

In [25]:
X_train.shape, X_val.shape

((27928, 1791), (6983, 1791))

In [26]:
embedding_vector = {}
with open('glove.840B.300d/glove.840B.300d.txt') as f:
    for line in tqdm(f):
        value = line.split(' ')
        word = value[0]
        coef = np.array(value[1:],dtype = 'float32')
        embedding_vector[word] = coef

2196017it [02:55, 12504.68it/s]


In [27]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, idx in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[idx] = embedding_value

100%|██████████| 58761/58761 [00:00<00:00, 392022.02it/s]


# Modeling

In [28]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping #ModelCheckpoint, Callback, 

In [33]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=1791, trainable = False))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(15,activation = 'softmax'))

In [34]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:
early = EarlyStopping(monitor="val_accuracy", mode='max', patience=5, restore_best_weights=True)

In [36]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 50, batch_size=256, callbacks=[early]) 

Epoch 1/50
110/110 [==============================] - 141s 1s/step - loss: 1.6724 - accuracy: 0.5199 - val_loss: 1.0835 - val_accuracy: 0.7014
Epoch 2/50
110/110 [==============================] - 132s 1s/step - loss: 1.0108 - accuracy: 0.7297 - val_loss: 0.8629 - val_accuracy: 0.7659
Epoch 3/50
110/110 [==============================] - 132s 1s/step - loss: 0.8188 - accuracy: 0.7927 - val_loss: 0.6857 - val_accuracy: 0.8157
Epoch 4/50
110/110 [==============================] - 132s 1s/step - loss: 0.7107 - accuracy: 0.8192 - val_loss: 0.6372 - val_accuracy: 0.8296
Epoch 5/50
110/110 [==============================] - 131s 1s/step - loss: 0.6944 - accuracy: 0.8163 - val_loss: 0.6192 - val_accuracy: 0.8317
Epoch 6/50
110/110 [==============================] - 131s 1s/step - loss: 0.6037 - accuracy: 0.8398 - val_loss: 0.5651 - val_accuracy: 0.8448
Epoch 7/50
110/110 [==============================] - 131s 1s/step - loss: 0.5664 - accuracy: 0.8482 - val_loss: 0.5216 - val_accuracy: 0.8555

In [37]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

columns = ['Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 'Recall-MacroAVG', 'Recall-WeightedAVG', 
           'F1_score-MacroAVG','F1_score-WeightedAVG']

y_pred = model.predict(X_val)
y_pred = np.argmax(y_pred, axis=1)

record = {}
record['Accuracy'] = accuracy_score(y_val, y_pred)*100
record['Precision-MacroAVG'] = precision_score(y_val, y_pred, average='macro')*100
record['Precision-WeightedAVG'] = precision_score(y_val, y_pred, average='weighted')*100
record['Recall-MacroAVG'] = recall_score(y_val, y_pred, average='macro')*100
record['Recall-WeightedAVG'] = recall_score(y_val, y_pred, average='weighted')*100
record['F1_score-MacroAVG'] = f1_score(y_val, y_pred, average='macro')*100
record['F1_score-WeightedAVG'] = f1_score(y_val, y_pred, average='weighted')*100

results = pd.Series(record, index=columns)

In [38]:
results

Accuracy                 92.581985
Precision-MacroAVG       87.568073
Precision-WeightedAVG    92.731386
Recall-MacroAVG          87.792831
Recall-WeightedAVG       92.581985
F1_score-MacroAVG        87.577422
F1_score-WeightedAVG     92.596061
dtype: float64

In [39]:
model.save('DeepApproach.h5')

# Testing on test_data

In [40]:
test_data = pd.read_csv('test_in_kag_org.csv')

In [41]:
test_data.shape

(6501, 4)

In [42]:
test_data.isnull().sum()

title         0
text          0
title_text    0
label         0
dtype: int64

In [43]:
test_data.drop(columns=['title', 'text'], inplace=True)

# Preparing our data for modeling 

In [44]:
# # text
# text_corpus_test = []
# lm = WordNetLemmatizer()
# words = stopwords.words('english')

# for i in range(test_data.shape[0]):
#     row = re.sub('[^a-zA-Z\s]', '', test_data.iloc[i, 0]).lower()
#     row = row.split()  
#     row = [lm.lemmatize(word) for word in row if word not in words]
#     row = ' '.join(row)
#     text_corpus_test.append(row)

In [45]:
# len(text_corpus_test)

In [46]:
# text_corpus_test[0]

_Our Bag of Words_

In [54]:
# X_text_test = token.texts_to_sequences(text_corpus_test)
X_text_test = token.texts_to_sequences(test_data.iloc[:, 0].values)
pad_seq_test = pad_sequences(X_text_test, maxlen=1791)

In [55]:
X_test = pad_seq_test.copy()
y_test = test_data.iloc[:, 1].values

In [56]:
X_test.shape, y_test.shape

((6501, 1791), (6501,))

In [57]:
y_test = le.transform(y_test)

In [58]:
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [59]:
tf.keras.models.load_model('DeepApproach.h5')

In [60]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

record = {}
record['Accuracy'] = accuracy_score(y_test, y_pred)*100
record['Precision-MacroAVG'] = precision_score(y_test, y_pred, average='macro')*100
record['Precision-WeightedAVG'] = precision_score(y_test, y_pred, average='weighted')*100
record['Recall-MacroAVG'] = recall_score(y_test, y_pred, average='macro')*100
record['Recall-WeightedAVG'] = recall_score(y_test, y_pred, average='weighted')*100
record['F1_score-MacroAVG'] = f1_score(y_test, y_pred, average='macro')*100
record['F1_score-WeightedAVG'] = f1_score(y_test, y_pred, average='weighted')*100

results = pd.Series(record, index=columns)
results

Accuracy                 90.416859
Precision-MacroAVG       67.099412
Precision-WeightedAVG    90.813350
Recall-MacroAVG          70.025822
Recall-WeightedAVG       90.416859
F1_score-MacroAVG        68.169415
F1_score-WeightedAVG     90.523898
dtype: float64